# Лекция 6. Метрики для задачи регрессии

In [ ]:
from IPython.display import Image
import pandas as pd #pip install pandas/openpyxl
import plotly.express as px #pip install plotly
import statsmodels.formula.api as smf #pip install statsmodels
import re
from sklearn.model_selection import train_test_split #pip install scikit-learn
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import plotly.graph_objects as go
import numpy as np #pip install numpy
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
import math

In [ ]:
#что из себя представляет задача регрессии?

In [32]:
laptops=pd.read_csv('data\laptop_price.csv',encoding='Windows-1251')
laptops.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60


In [33]:
laptops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         1303 non-null   int64  
 1   Company           1303 non-null   object 
 2   Product           1303 non-null   object 
 3   TypeName          1303 non-null   object 
 4   Inches            1303 non-null   float64
 5   ScreenResolution  1303 non-null   object 
 6   Cpu               1303 non-null   object 
 7   Ram               1303 non-null   object 
 8   Memory            1303 non-null   object 
 9   Gpu               1303 non-null   object 
 10  OpSys             1303 non-null   object 
 11  Weight            1303 non-null   object 
 12  Price_euros       1303 non-null   float64
dtypes: float64(2), int64(1), object(10)
memory usage: 132.5+ KB


Результирующая переменная - Price_euros. Будем предсказывать цену ноутбука по его характеристикам.

In [34]:
#предобработка
laptops['Weight']=laptops['Weight'].str.replace('kg','').astype('float64')
ssd=[]
hdd=[]
fs=[]
for i in laptops['Memory']:
    if 'SSD' in i:
        ssd.append(1)
        hdd.append(0)
        fs.append(0)
    elif 'HDD' in i:
        ssd.append(0)
        hdd.append(1)
        fs.append(0)
    else:
        ssd.append(0)
        hdd.append(0)
        fs.append(1)
laptops['ssd']=ssd
laptops['hdd']=hdd
laptops['fs']=fs
s = [float(re.findall(r'\d+', s)[0]) for s in laptops['Memory']]
laptops['Memory']=s
laptops.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros,ssd,hdd,fs
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128.0,Intel Iris Plus Graphics 640,macOS,1.37,1339.69,1,0,0
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128.0,Intel HD Graphics 6000,macOS,1.34,898.94,0,0,1
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256.0,Intel HD Graphics 620,No OS,1.86,575.00,1,0,0
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512.0,AMD Radeon Pro 455,macOS,1.83,2537.45,1,0,0
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256.0,Intel Iris Plus Graphics 650,macOS,1.37,1803.60,1,0,0


In [35]:
#реализация модели с помощью statsmodels
model = smf.ols(formula='Price_euros ~ Weight + Inches + Memory + Weight + ssd + hdd + fs', data=laptops).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            Price_euros   R-squared:                       0.370
Model:                            OLS   Adj. R-squared:                  0.367
Method:                 Least Squares   F-statistic:                     152.1
Date:                Fri, 10 Mar 2023   Prob (F-statistic):          3.26e-127
Time:                        13:44:30   Log-Likelihood:                -10082.
No. Observations:                1303   AIC:                         2.018e+04
Df Residuals:                    1297   BIC:                         2.021e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1201.8446    181.442      6.624      0.0

##метрика 1 - коэффициент детерминации

In [36]:
Image(url="https://image3.slideserve.com/5567030/slide41-l.jpg")

Коэффициент детерминации (Coefficient of determination) показывает долю дисперсии зависимой переменной, объяснённой с помощью регрессионной модели. Когда значение коэффициента близко к 0 (т.е. ошибка модели с переменными примерно равна ошибке модели только с константой), это указывает на низкое соответствие модели данным, когда модель с переменными работает не лучше модели с константой.
Кроме этого, бывают ситуации, когда коэффициент R2
  принимает отрицательные значения (обычно небольшие). Это произойдёт, если ошибка модели среднего становится меньше ошибки модели с переменной. В этом случае оказывается, что добавление в модель с константой некоторой переменной только ухудшает её (т.е. регрессионная модель с переменной работает хуже, чем предсказание с помощью простой средней).
На практике используют следующую шкалу оценок. Модель, для которой R2>0.5, является удовлетворительной. Если R2>0.8, то модель рассматривается как очень хорошая. Значения, меньшие 0.5 говорят о том, что модель плохая.

In [37]:
laptops_lr = laptops[['Inches','Memory','Weight','ssd','hdd','fs','Price_euros']]
X=laptops_lr.drop(['Price_euros'],axis=1)
Y=laptops_lr['Price_euros']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
regressor = LinearRegression()
regressor.fit(X_train, y_train) # А тут обучение на обучающей выборке

LinearRegression()

In [44]:
#расчет
y_pred=regressor.predict(X_test) #предсказываем значения цены для тестовой выборки
round(r2_score(y_test, y_pred),2)

0.32

## метрика 2 - среднеквадратическая ошибка

Среднеквадратичная ошибка (Mean Squared Error) применяется в случаях, когда требуется подчеркнуть большие ошибки и выбрать модель, которая дает меньше именно больших ошибок. Большие значения ошибок становятся заметнее за счет квадратичной зависимости.

Действительно, допустим модель допустила на двух примерах ошибки 5 и 10. В абсолютном выражении они отличаются в два раза, но если их возвести в квадрат, получив 25 и 100 соответственно, то отличие будет уже в четыре раза. Таким образом модель, которая обеспечивает меньшее значение MSE допускает меньше именно больших ошибок.

In [45]:
Image(url="https://vitalflux.com/wp-content/uploads/2021/08/mean-squared-error-loss.png",height=200,width=500)

Корень из среднеквадратичной ошибки (Root Mean Squared Error) вычисляется просто как квадратный корень. Влияние каждой ошибки на RMSE пропорционально величине квадрата ошибки. Поэтому большие ошибки оказывают непропорционально большое влияние на RMSE. Следовательно, RMSE можно считать чувствительной к аномальным значениям.

In [46]:
#расчет
mse = mean_squared_error(y_test, y_pred)
print(f'среднеквадратическая ошибка = {round(mse,2)}')
print(f'корень из mse = {round(math.sqrt(mse),2)}')

среднеквадратическая ошибка = 397853.71
корень из mse = 630.76


## метрика 3 - средняя абсолютная ошибка

MAE рассчитывается как среднее абсолютных разностей между наблюдаемым и предсказанным значениями. В отличие от MSE и RMSE она является линейной оценкой, а это значит, что все ошибки в среднем взвешены одинаково. Например, разница между 0 и 10 будет вдвое больше разницы между 0 и 5. Для MSE и RMSE, как отмечено выше, это не так.

Поэтому MAE широко используется, например, в финансовой сфере, где ошибка в 10 долларов должна интерпретироваться как в два раза худшая, чем ошибка в 5 долларов.

In [47]:
Image(url="https://img-blog.csdnimg.cn/20181118213345875.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L0NvZGVyUGFp,size_16,color_FFFFFF,t_70",height=200,width=500)

In [48]:
round(mean_absolute_error(y_test, y_pred),2)

448.51

## метрика 4 - средняя абсолютная процентная ошибка

In [49]:
Image(url="https://botana.biz/prepod/_bloks/pic/gl1y5zb-061.png")

Эта ошибка не имеет размерности и очень проста в интерпретации. Её можно выражать как в долях, так и в процентах. Если получилось, например, что MAPE=11.4, то это говорит о том, что ошибка составила 11.4% от фактического значения.

In [50]:
round(mean_absolute_percentage_error(y_test, y_pred),2)

0.48

In [51]:
#запишем метрики модели в отдельный список
m_model1 = [round(r2_score(y_test, y_pred),2),round(mse,2),round(math.sqrt(mse),2),round(mean_absolute_error(y_test, y_pred),2),round(mean_absolute_percentage_error(y_test, y_pred),2)]

In [52]:
m_model1

[0.32, 397853.71, 630.76, 448.51, 0.48]

In [53]:
#сделаем вторую модель, в которую добавим объем оперативной памяти
laptops['Ram']=laptops['Ram'].str.replace('GB','').astype('int32')

In [54]:
laptops.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros,ssd,hdd,fs
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,128.0,Intel Iris Plus Graphics 640,macOS,1.37,1339.69,1,0,0
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,128.0,Intel HD Graphics 6000,macOS,1.34,898.94,0,0,1
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256.0,Intel HD Graphics 620,No OS,1.86,575.00,1,0,0
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,512.0,AMD Radeon Pro 455,macOS,1.83,2537.45,1,0,0
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,256.0,Intel Iris Plus Graphics 650,macOS,1.37,1803.60,1,0,0


In [55]:
laptops_lr2 = laptops[['Inches','Memory','Weight','ssd','hdd','fs','Ram','Price_euros']]
X=laptops_lr2.drop(['Price_euros'],axis=1)
Y=laptops_lr2['Price_euros']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
regressor2 = LinearRegression()
regressor2.fit(X_train, y_train)
y_pred=regressor2.predict(X_test)

In [56]:
#расчет метрик для второй модели
r2_m2 = round(r2_score(y_test, y_pred),2)
mse_m2 = round(mean_squared_error(y_test, y_pred),2)
rmse_m2 = round(math.sqrt(mean_squared_error(y_test, y_pred)),2)
mae_m2 = round(mean_absolute_error(y_test, y_pred),2)
mape_m2 = round(mean_absolute_percentage_error(y_test, y_pred),2)
m_model2 = [r2_m2,mse_m2,rmse_m2,mae_m2,mape_m2]
m_model2

[0.6, 235224.07, 485.0, 335.42, 0.32]

In [57]:
metrics = pd.DataFrame(columns=['metric','m1','m2'])
metrics['metric'] = ['r2','mse','rmse','mae','mape']
metrics['m1'] = m_model1
metrics['m2'] = m_model2
metrics

,metric,m1,m2
0,r2,0.32,0.60
1,mse,397853.71,235224.07
2,rmse,630.76,485.00
3,mae,448.51,335.42
4,mape,0.48,0.32


## Реализация лассо и ридж-регрессии


In [58]:
model_lasso = Lasso(alpha=0.5)
model_ridge = Ridge(alpha=0.5)
model_lasso.fit(X_train,y_train)
model_ridge.fit(X_train,y_train)
y_pred_lasso = model_lasso.predict(X_test)
y_pred_ridge = model_ridge.predict(X_test)

In [59]:
print(f'mape lasso = {round(mean_absolute_percentage_error(y_test, y_pred_lasso),2)}')
print(f'mape ridge = {round(mean_absolute_percentage_error(y_test, y_pred_ridge),2)}')

mape lasso = 0.32
mape ridge = 0.32


In [61]:
for a in [0.01, 0.05, 0.1]:
    model_lasso = Lasso(alpha=a)
    model_ridge = Ridge(alpha=a)
    model_lasso.fit(X_train,y_train)
    model_ridge.fit(X_train,y_train)
    y_pred_lasso = model_lasso.predict(X_test)
    y_pred_ridge = model_ridge.predict(X_test)
    print(f'mape lasso({a}) = {round(mean_absolute_percentage_error(y_test, y_pred_lasso),2)}')
    print(f'mape ridge({a}) = {round(mean_absolute_percentage_error(y_test, y_pred_ridge),2)}')

mape lasso(0.01) = 0.32
mape ridge(0.01) = 0.32
mape lasso(0.05) = 0.32
mape ridge(0.05) = 0.32
mape lasso(0.1) = 0.32
mape ridge(0.1) = 0.32


In [62]:
true_vs_pred = pd.DataFrame({'Истинные': y_test, 'Предсказанные': y_pred_lasso})
true_vs_pred.head()

,Истинные,Предсказанные
580,389.00,788.747137
1017,2968.00,1838.648913
761,1859.00,1986.873240
686,612.61,606.036200
186,2397.00,1942.762071


In [63]:
random_x = np.linspace(0, len(true_vs_pred['Истинные']))

fig = go.Figure()
fig.add_trace(go.Scatter(x=random_x, y=true_vs_pred['Истинные'],
                    mode='lines',
                    name='true'))
fig.add_trace(go.Scatter(x=random_x, y=true_vs_pred['Предсказанные'],
                    mode='lines+markers',
                    name='pred'))

fig.show()

## Лабораторная работа №2

* ищем набор данных, для которого можно решить задачу регрессии (критерии к данным такие же, как в ЛР1)
* провести предобработку признаков и их последующий отбор
* строим модели - минимум 3 различные модели (разный набор признаков, разные алгоритмы)
* с помощью метрик оценить качество полученных моделей на тестовой выборке, выбрать наилучшую
* построить график для тестовых значений (предсказанные с помощью лучшей модели против реальных)

могут быть допбаллы за хорошую многомерную модель